In [ ]:
!apt-get install -y fluidsynth
!pip install pyfluidsynth pretty_midi

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
# third party imports
import os
import sys
import numpy as np
import matplotlib.pyplot as plt

# adding preprocessing directory to path
current_dir = os.path.dirname(os.path.abspath("preprocessing"))
parent_dir = os.path.join(current_dir, '..')
sys.path.append(parent_dir)

from preprocessing.audio_to_spectrogram import (
    reducer_sampler,
    midi_notes_dataframe,
    midi_to_array,
    midi_track_length,
    midi_to_spectrogram,
    spectrogram_sample)

# Simple Array Represenation generation pipeline example

In [ ]:
# The following code demonstrates the process for generating
# and sampling SRA spectrograms.
base_path = r"../data/midi/"
sub_dirs = ["Bach", "Beethoven", "Chopin", "Mozart"]

# number of seconds to sample
sample_seconds = 3
# maximum allowable silence percentage
silence_threshold = 0.20 
# samples to pull per midi
sample_times = 4

# loop through each composer directory
for dir in sub_dirs:
    path = f"{base_path}{dir}/"
    files = os.listdir(path)

    for _ in range(250):
        # choose a random file
        file = np.random.choice(files)
        file_path = path + file

        try:
            # extract notes
            midi_df = midi_notes_dataframe(file_path)
            # convert to array representation
            array = midi_to_array(midi_df)
            # extract track length in seconds for scaling
            duration = midi_track_length(file_path)
        except Exception as e:
            print(e)
            pass
        for i in range(sample_times):
            try:
                # set output location
                dest_path = f"../data/spectrograms/{dir}_1/{file[:-4]}_{start}_{stop}.png"
                print(dest_path)
                # sample a random segment of length sample_seconds
                start, stop, sample = reducer_sampler(
                    array, duration, sample_seconds, silence_threshold)

                plt.imsave(dest_path, sample)
                print("success")
            except Exception as e:
                print(e)
                pass

# Mel Spectrogram Generation Pipeline Example

In [ ]:
# The following code demonstrates the process for generating
# and sampling Mel spectrograms.
base_midi_path = r"/content/drive/MyDrive/midi/"
sub_dirs = ["Bach", "Beethoven", "Chopin", "Mozart"]
# location of soundfont file
soundfont = "/content/drive/MyDrive/FluidR3_GM/FluidR3_GM.sf2"
base_output = "/content/drive/MyDrive/mel_spectrograms_3sec"

sample_seconds = 3
sample_times = 4
samples = 0

# loop through each composer directory
for dir in sub_dirs:
    path = f"{base_midi_path}{dir}/"
    files = os.listdir(path)

    for _ in range(75):
        # choose a random file
        file = np.random.choice(files)
        file_path = path + file
        print(file_path)
        try:
            # convert midi file to mel spectrogram
            array, sr = midi_to_spectrogram(file_path, soundfont)
        except Exception as e:
            print(e)
            pass
        for i in range(sample_times):
            try:
                # extract sample from mel spectrogram
                start, stop, sample = spectrogram_sample(
                    array, sr, sample_seconds)
                dest_path = f"{base_output}/{dir}/{file[:-4]}_{start}_{stop}.png"
                plt.imsave(dest_path, sample)
                samples += 1
                print("success ", samples)
            except Exception as e:
                print(e)
                pass